In [35]:
import csv
import math

def read_sites_from_file(filename):
    sites = []
    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            sites.append({
                'site_id': row['site_id'],
                'latitude': float(row['latitude']),
                'longitude': float(row['longitude'])
            })
    return sites

def find_closest_site_id(sites, lat, lon):
    min_distance = float('inf')
    closest_site_id = None

    for site in sites:
        distance = math.sqrt((lat - site['latitude'])**2 + (lon - site['longitude'])**2)
        if distance < min_distance:
            min_distance = distance
            closest_site_id = site['site_id']

    return closest_site_id

def main():
    sites = read_sites_from_file('wtk_site_metadata.csv')

    nodes = {
    "CT-REV": (41.149972, -71.069972),
    "RI-REV": (41.149972, -71.069972),
    "VINE": (41.03325, -70.61667),
    "PKCTY": (40.859972, -70.709972),
    "COMW": (40.77634, -70.695545),
    "MFLR1": (40.797153, -70.616667),
    "MFLR2": (40.806968, -70.547001),
    }

    matched_sites = []
    for location, (lat, lon) in nodes.items():
        site_id = find_closest_site_id(sites, lat, lon)
        print(f"Closest site_id for {location} (latitude {lat} and longitude {lon}) is: {site_id}")
        matched_sites.append((location, site_id))
        
    print(matched_sites)
        
  
        # Writing matched sites to a new CSV file
    with open('wtk_site_metadata.csv', 'r') as input_csv, open('wtk_site_metadata_matched.csv', 'w', newline='') as output_csv:
        reader = csv.DictReader(input_csv)
        writer = csv.DictWriter(output_csv, fieldnames=reader.fieldnames)
        writer.writeheader()

        for row in reader:
            for site in [int(site_id) for _, site_id in matched_sites]:
                if int(row['site_id']) == site:
                    writer.writerow(row)

if __name__ == '__main__':
    main()


Closest site_id for CT-REV (latitude 41.149972 and longitude -71.069972) is: 94658
Closest site_id for RI-REV (latitude 41.149972 and longitude -71.069972) is: 94658
Closest site_id for VINE (latitude 41.03325 and longitude -70.61667) is: 96008
Closest site_id for PKCTY (latitude 40.859972 and longitude -70.709972) is: 94663
Closest site_id for COMW (latitude 40.77634 and longitude -70.695545) is: 94663
Closest site_id for MFLR1 (latitude 40.797153 and longitude -70.616667) is: 96008
Closest site_id for MFLR2 (latitude 40.806968 and longitude -70.547001) is: 93857
[('CT-REV', '94658'), ('RI-REV', '94658'), ('VINE', '96008'), ('PKCTY', '94663'), ('COMW', '94663'), ('MFLR1', '96008'), ('MFLR2', '93857')]


In [36]:
conda install -c anaconda netcdf4

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.9.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.9.0



## Package Plan ##

  environment location: /Users/skhanal/anaconda3

  added / updated specs:
    - netcdf4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.08.22 |       hca03da5_0         131 KB  anaconda
    certifi-2023.7.22          |  py311hca03da5_0         157 KB  anaconda
    cftime-1.6.2               |  py311ha0d4635_0         218 KB  anaconda
    conda-23.9.0               |  py311hca03da5_0         1.3 MB  anaconda
    hdf4-4.2.13                |       h5e329fb_3         892 KB  anaconda
    libzip-1.8.0 

In [40]:
import csv
import os
from netCDF4 import Dataset

def read_nc_file(file_path):
    """Read the netCDF file and return its data."""
    with Dataset(file_path, 'r') as nc_file:
        # Here we are assuming all variables in the netCDF file should be read.
        # If you only want specific variables, you can modify this section.
        return {var: nc_file.variables[var][:].tolist() for var in nc_file.variables}

def main():
    # List of your netCDF files
    nc_files = ['93857.nc', '96008.nc', '94663.nc', '94658.nc']

    data_dicts = [read_nc_file(file) for file in nc_files]
    headers = [os.path.splitext(file)[0] for file in nc_files]

    # Assume all netCDF files have the same variables and length
    variable_names = list(data_dicts[0].keys())
    data_length = len(data_dicts[0][variable_names[0]])

    with open('wind_out.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        # Write headers
        writer.writerow(['Variable'] + headers)

        for var in variable_names:
            for i in range(data_length):
                writer.writerow([var] + [data_dict[var][i] for data_dict in data_dicts])

if __name__ == '__main__':
    main()
